In [0]:
!pip install fasttext

In [0]:
from google.colab import drive
import fasttext
import numpy as np
import pandas as pd
import datetime
import warnings
drive.mount('/gdrive')
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Models/cc.lv.300.bin'
warnings.filterwarnings('ignore')
ft = fasttext.load_model('cc.lv.300.bin')
max_words = len(ft.get_words())
print('Vārdnīcas izmērs:',max_words)
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets'

In [0]:
eval_data = pd.read_excel('izvertesanas_jautajumu_kopa.xlsx')

In [0]:
encoded_eval = pd.read_csv('2020_05_24_encoded_Fasttext_testa_kopa.csv', index_col=0)
print(encoded_eval.shape)
X = encoded_eval.iloc[:,:300]
Y = encoded_eval.iloc[:,300:]
print(X.shape, Y.shape)

(100, 301)
(100, 300) (100, 1)


In [0]:
def transformResults(results):
    mod_results = []
    for result in results:
        out = np.argmax(result)
        mod_results.append(out)
    mod_results = np.array(mod_results)    
    return mod_results 

<h2>Nepapildināto jautājumu datu kopas izvērtēšana</h2>

In [0]:
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets'
fasttext_df = pd.read_csv('2020_05_19_encoded_Fasttext_sakotnejie.csv', index_col=0)
col_count = len(fasttext_df.columns) - 1
features = fasttext_df.iloc[:,:col_count]
target = fasttext_df.iloc[:,col_count:]
print(features.shape, target.shape)

/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets
(23217, 300) (23217, 1)


In [0]:
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(features, target,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                     # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                       # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)             # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.31


In [0]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)         # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.3413
              precision    recall  f1-score   support

  akadēmisks       0.00      0.00      0.00        25
        citi       0.31      1.00      0.48        25
 darba laiks       1.00      0.80      0.89        25
   dokumenti       0.00      0.00      0.00        25

    accuracy                           0.45       100
   macro avg       0.33      0.45      0.34       100
weighted avg       0.33      0.45      0.34       100

[[ 0 25  0  0]
 [ 0 25  0  0]
 [ 0  5 20  0]
 [ 0 25  0  0]]
[[0.  1.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.2 0.8 0. ]
 [0.  1.  0.  0. ]]


<h2>Papildināto jautājumu izvērtēšana</h2>

In [0]:
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets'
papildinatie = pd.read_csv('2020_05_09_encoded_Fasttext.csv', index_col=0)
col_count = len(papildinatie.columns) - 1
features = papildinatie.iloc[:,:col_count]
target = papildinatie.iloc[:,col_count:]
print(features.shape, target.shape)

/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets
(23391, 300) (23391, 1)


In [0]:
papildinatie['labels'].value_counts()

1    23025
3      122
2      122
0      122
Name: labels, dtype: int64

In [0]:
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(features, target,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.78


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.7944
              precision    recall  f1-score   support

  akadēmisks       0.92      0.92      0.92        25
        citi       0.57      0.92      0.71        25
 darba laiks       1.00      0.60      0.75        25
   dokumenti       0.90      0.72      0.80        25

    accuracy                           0.79       100
   macro avg       0.85      0.79      0.79       100
weighted avg       0.85      0.79      0.79       100

[[23  2  0  0]
 [ 0 23  0  2]
 [ 1  9 15  0]
 [ 1  6  0 18]]
[[0.92 0.08 0.   0.  ]
 [0.   0.92 0.   0.08]
 [0.04 0.36 0.6  0.  ]
 [0.04 0.24 0.   0.72]]


<h2>UnderSampling izvērtēšana</h2>

In [0]:
from imblearn.under_sampling import NearMiss
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = undersampler.fit_resample(features, target)                   # Veicam Unersampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
(23391, 300) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.66


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.6885
              precision    recall  f1-score   support

  akadēmisks       0.64      1.00      0.78        25
        citi       0.67      0.16      0.26        25
 darba laiks       0.85      0.88      0.86        25
   dokumenti       0.79      0.92      0.85        25

    accuracy                           0.74       100
   macro avg       0.74      0.74      0.69       100
weighted avg       0.74      0.74      0.69       100

[[25  0  0  0]
 [11  4  4  6]
 [ 2  1 22  0]
 [ 1  1  0 23]]
[[1.   0.   0.   0.  ]
 [0.44 0.16 0.16 0.24]
 [0.08 0.04 0.88 0.  ]
 [0.04 0.04 0.   0.92]]


<h2>OverSampling izvērtēšana</h2>

<h3>OverSampling izmērs = 100%</h3>


In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 2*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
(23391, 300) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.81


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.7952
              precision    recall  f1-score   support

  akadēmisks       0.96      0.92      0.94        25
        citi       0.56      0.96      0.71        25
 darba laiks       1.00      0.56      0.72        25
   dokumenti       0.95      0.72      0.82        25

    accuracy                           0.79       100
   macro avg       0.87      0.79      0.80       100
weighted avg       0.87      0.79      0.80       100

[[23  2  0  0]
 [ 0 24  0  1]
 [ 1 10 14  0]
 [ 0  7  0 18]]
[[0.92 0.08 0.   0.  ]
 [0.   0.96 0.   0.04]
 [0.04 0.4  0.56 0.  ]
 [0.   0.28 0.   0.72]]


<h3>OverSampling izmērs = 200%</h3>


In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 3*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
(23391, 300) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.82


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.8249
              precision    recall  f1-score   support

  akadēmisks       1.00      0.96      0.98        25
        citi       0.59      0.92      0.72        25
 darba laiks       1.00      0.60      0.75        25
   dokumenti       0.91      0.80      0.85        25

    accuracy                           0.82       100
   macro avg       0.87      0.82      0.82       100
weighted avg       0.87      0.82      0.82       100

[[24  1  0  0]
 [ 0 23  0  2]
 [ 0 10 15  0]
 [ 0  5  0 20]]
[[0.96 0.04 0.   0.  ]
 [0.   0.92 0.   0.08]
 [0.   0.4  0.6  0.  ]
 [0.   0.2  0.   0.8 ]]


<h3>OverSampling izmērs = 300%</h3>


In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 4*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
(23391, 300) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.82


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('FastText model score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

(23391, 300) (23391, 1)
[[    0     1     2     3]
 [  488 23025   488   488]]
Keras Done
FastText model score: 0.8002
              precision    recall  f1-score   support

  akadēmisks       1.00      0.84      0.91        25
        citi       0.55      0.92      0.69        25
 darba laiks       1.00      0.60      0.75        25
   dokumenti       0.91      0.80      0.85        25

    accuracy                           0.79       100
   macro avg       0.86      0.79      0.80       100
weighted avg       0.86      0.79      0.80       100

[[21  4  0  0]
 [ 0 23  0  2]
 [ 0 10 15  0]
 [ 0  5  0 20]]
[[0.84 0.16 0.   0.  ]
 [0.   0.92 0.   0.08]
 [0.   0.4  0.6  0.  ]
 [0.   0.2  0.   0.8 ]]


<h3>Pareizo, nepareizo jautājumu paredzēšana un izdrukāšana</h3>

In [0]:
jautajumi = pd.read_excel('izvertesanas_jautajumu_kopa.xlsx')

<h3>Visu testa jautājumu atbilžu izdrukāšana</h3>

In [0]:
jaut = jautajumi['Jautājums'].to_numpy()
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)
counter = 0
for p in y_pred:
  print('Jautājums:', jaut[counter])
  if keras_y_pred[counter] == 0:
    print('Kategorija: Akadēmisks')
  elif keras_y_pred[counter] == 1:
    print('Kategorija: Citi')
  elif keras_y_pred[counter] == 2:
    print('Kategorija: Darba laiks')
  elif keras_y_pred[counter] == 3:
    print('Kategorija: Dokumenti')
  print('Klases:         [akademisks, citi, darba laiks, dokumenti]')  
  print('Klašu varbūtības:', p)
  counter = counter + 1

Jautājums: Kas jādara, lai paņemtu akadēmisko gadu?
Kategorija: Akadēmisks
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.999 0.001 0.    0.   ]
Jautājums: Kādi dokumenti jāiesniedz, lai dotos ERASMUS?
Kategorija: Dokumenti
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.    0.001 0.    0.999]
Jautājums: Kāds ir LU DAC darba laiks brīvdienās?
Kategorija: Darba laiks
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.    0.002 0.998 0.   ]
Jautājums: Līdz cikiem strādā bibliotēka?
Kategorija: Darba laiks
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.    0.006 0.994 0.   ]
Jautājums: Ar ko jāsaskaņo reklāmas materiālu izvietošana LU telpās?
Kategorija: Dokumenti
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.003 0.066 0.    0.931]
Jautājums: Cikos tiek atvērta universitātes galvenā ēka?
Kategorija: Darba laiks
Klases:         [ak

<h3>Nepareizi klasificēto jautājumu izdrukāšana</h3>

In [0]:
jaut = jautajumi['Jautājums'].to_numpy()
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)
counter = 0
false_count = 0
for p in y_pred:
  if keras_y_pred[counter] != y_test[counter]:
    false_count = false_count + 1
    print('Jautājums:', jaut[counter])
    if y_test[counter] == 0:
      print('Pareizā kategorija: Akadēmisks')
    elif y_test[counter] == 1:
      print('Pareizā kategorija: Citi')
    elif y_test[counter] == 2:
      print('Pareizā kategorija: Darba laiks')
    elif y_test[counter] == 3:
      print('Pareizā kategorija: Dokumenti')
      
    if keras_y_pred[counter] == 0:
      print('Noteiktā kategorija: Akadēmisks')
    elif keras_y_pred[counter] == 1:
      print('Noteiktā kategorija: Citi')
    elif keras_y_pred[counter] == 2:
      print('Noteiktā kategorija: Darba laiks')
    elif keras_y_pred[counter] == 3:
      print('Noteiktā kategorija: Dokumenti')
    print('Klases:         [akademisks, citi, darba laiks, dokumenti]')  
    print('Klašu varbūtības:', p)
  counter = counter + 1
print('Neprecīzi noteikto jautājumu skaits:',false_count, 'no 100')  

Jautājums: Ar ko jāsaskaņo reklāmas materiālu izvietošana LU telpās?
Pareizā kategorija: Citi
Noteiktā kategorija: Dokumenti
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.   0.07 0.   0.93]
Jautājums: Līdz cikiem strādā LU DF dekanāts?
Pareizā kategorija: Darba laiks
Noteiktā kategorija: Citi
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.   0.71 0.29 0.  ]
Jautājums: Vai 1. septembrī strādās LUSP birojs?
Pareizā kategorija: Darba laiks
Noteiktā kategorija: Citi
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.   0.71 0.29 0.  ]
Jautājums: Cik ilgi drīkst atrasties LU DACā?
Pareizā kategorija: Darba laiks
Noteiktā kategorija: Citi
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.01 0.97 0.01 0.  ]
Jautājums: Vai ir iespējams tikt pie apliecinājuma, ka esmu studējošais?
Pareizā kategorija: Dokumenti
Noteiktā kategorija: Citi
Klases:         [akademisks, citi, 

<h3>Pareizi noteikto jautājumu printēšana</h3>


In [0]:
jaut = jautajumi['Jautājums'].to_numpy()
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)
counter = 0
true_count = 0
for p in y_pred:
  if keras_y_pred[counter] == y_test[counter]:
    true_count = true_count + 1
    print('Jautājums:', jaut[counter])
    if y_test[counter] == 0:
      print('Pareizā kategorija: Akadēmisks')
    elif y_test[counter] == 1:
      print('Pareizā kategorija: Citi')
    elif y_test[counter] == 2:
      print('Pareizā kategorija: Darba laiks')
    elif y_test[counter] == 3:
      print('Pareizā kategorija: Dokumenti')
      
    if keras_y_pred[counter] == 0:
      print('Noteiktā kategorija: Akadēmisks')
    elif keras_y_pred[counter] == 1:
      print('Noteiktā kategorija: Citi')
    elif keras_y_pred[counter] == 2:
      print('Noteiktā kategorija: Darba laiks')
    elif keras_y_pred[counter] == 3:
      print('Noteiktā kategorija: Dokumenti')
    print('Klases:         [akademisks, citi, darba laiks, dokumenti]')  
    print('Klašu varbūtības:', p)
  counter = counter + 1
print('Precīzi noteikto jautājumu skaits:',true_count, 'no 100')  

Jautājums: Kas jādara, lai paņemtu akadēmisko gadu?
Pareizā kategorija: Akadēmisks
Noteiktā kategorija: Akadēmisks
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [1. 0. 0. 0.]
Jautājums: Kādi dokumenti jāiesniedz, lai dotos ERASMUS?
Pareizā kategorija: Dokumenti
Noteiktā kategorija: Dokumenti
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0. 0. 0. 1.]
Jautājums: Kāds ir LU DAC darba laiks brīvdienās?
Pareizā kategorija: Darba laiks
Noteiktā kategorija: Darba laiks
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0. 0. 1. 0.]
Jautājums: Līdz cikiem strādā bibliotēka?
Pareizā kategorija: Darba laiks
Noteiktā kategorija: Darba laiks
Klases:         [akademisks, citi, darba laiks, dokumenti]
Klašu varbūtības: [0.   0.01 0.99 0.  ]
Jautājums: Cikos tiek atvērta universitātes galvenā ēka?
Pareizā kategorija: Darba laiks
Noteiktā kategorija: Darba laiks
Klases:         [akademisks, citi, darba laiks, dok

<h2>Kombinētā Under & Over Sampling izvērtēšana</h2>

<h3>OverSampling izmērs = 100%</h3>


In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 2*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
(23391, 300) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.63


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.6554
              precision    recall  f1-score   support

  akadēmisks       0.60      1.00      0.75        25
        citi       1.00      0.04      0.08        25
 darba laiks       0.92      0.96      0.94        25
   dokumenti       0.77      0.96      0.86        25

    accuracy                           0.74       100
   macro avg       0.82      0.74      0.66       100
weighted avg       0.82      0.74      0.66       100

[[25  0  0  0]
 [15  1  2  7]
 [ 1  0 24  0]
 [ 1  0  0 24]]
[[1.   0.   0.   0.  ]
 [0.6  0.04 0.08 0.28]
 [0.04 0.   0.96 0.  ]
 [0.04 0.   0.   0.96]]


<h3>OverSampling izmērs = 200%</h3>


In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 3*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
(23391, 300) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.63


In [0]:
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

[[  0   1   2   3]
 [122 122 122 122]]
[[  0   1   2   3]
 [366 122 366 366]]
Keras Done
Naive Bayes F1 score: 0.6511
              precision    recall  f1-score   support

  akadēmisks       0.64      1.00      0.78        25
        citi       1.00      0.04      0.08        25
 darba laiks       0.83      0.96      0.89        25
   dokumenti       0.77      0.96      0.86        25

    accuracy                           0.74       100
   macro avg       0.81      0.74      0.65       100
weighted avg       0.81      0.74      0.65       100

[[25  0  0  0]
 [12  1  5  7]
 [ 1  0 24  0]
 [ 1  0  0 24]]
[[1.   0.   0.   0.  ]
 [0.48 0.04 0.2  0.28]
 [0.04 0.   0.96 0.  ]
 [0.04 0.   0.   0.96]]


<h3>OverSampling izmērs = 300%</h3>


In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 4*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('FastText final score:',f1_final_score)

(100, 300) (100, 1)
(23391, 300) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
FastText final score: 0.64


In [0]:
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

[[  0   1   2   3]
 [122 122 122 122]]
[[  0   1   2   3]
 [488 122 488 488]]
Keras Done
Naive Bayes F1 score: 0.6347
              precision    recall  f1-score   support

  akadēmisks       0.69      1.00      0.82        25
        citi       0.00      0.00      0.00        25
 darba laiks       0.76      1.00      0.86        25
   dokumenti       0.77      0.96      0.86        25

    accuracy                           0.74       100
   macro avg       0.56      0.74      0.63       100
weighted avg       0.56      0.74      0.63       100

[[25  0  0  0]
 [10  0  8  7]
 [ 0  0 25  0]
 [ 1  0  0 24]]
[[1.   0.   0.   0.  ]
 [0.4  0.   0.32 0.28]
 [0.   0.   1.   0.  ]
 [0.04 0.   0.   0.96]]
